# Numerics Review
This notebook provides exercises to review our required math background and how to use `numpy` and `scipy`.
Useful documentation:
+ [numpy](https://numpy.org/doc/stable/reference/)
+ [scipy Linear Algebra](https://docs.scipy.org/doc/scipy/reference/tutorial/linalg.html)
+ [scipy Statistics](https://docs.scipy.org/doc/scipy/reference/stats.html)

In [ ]:
import numpy as np
import scipy.stats as stats
import scipy.linalg as linalg
# For pretty pictures!
import matplotlib.pyplot as plt

## Linear Algebra - Vectors and Matrices

`numpy` stores vectors and matrices in efficient array data structures. 
You build them using the `array()` constructor. 
Topics for review:
+ Matrices, vectors in code
+ Products
+ Inverse
+ Eigenvalues

In [ ]:
x1 = np.ones((2,1), dtype=np.float32)
x2 = np.array([1., 2.], dtype=np.float32).reshape(1,2)
A1 = np.array([[1., 2.],[3., 2.]], dtype=np.float32)

In [ ]:
x3 = x1.dot(x2)
print(x3)

In [ ]:
x4 = x2.dot(x1)
print(x4)

In [ ]:
y = A1.dot(x1)
print(y)

In [ ]:
z = A1.dot(A1)
print(z)

In [ ]:
A2 = np.array([[1., 2.],[2., 6.]])
A2inv = linalg.inv(A2)
print(A2inv.dot(A2))

In [ ]:
l,v = linalg.eig(A2)
print("Eigenvalues: {0}".format(l))
print("Eigenvectors: {0}".format(v))

## Probability and Stats

`numpy` and `scipy` provide useful APIs for modeling distributions and performing statistics on data arrays.
Use the code block below for in-class exercises.
Topics for review:
+ Data arrays
+ Mean, mode, median
+ Probability distributions

In [ ]:
d = np.array([.5, .55, .42, .46, .52, .36])

Various statistical functions over numpy arrays. Very useful!

In [ ]:
print(np.mean(d))
print(np.median(d))
print(np.std(d))
print(np.var(d))

Let's say our laser sensor has **zero mean** Gaussian noise with $\sigma=0.05$.
We can build a single variable Gaussian distribution that models the sensor.

In [ ]:
# A silly model of the laser measuring a wall 0.5 m away.
laser_model = stats.norm(loc=0.5, scale=0.05)
# Sample 5 measurements from this distribution
print(laser_model.rvs(5))

In [ ]:
fig, ax = plt.subplots(1, 1)
#fig.set_size_inches((5,4))
# PDF
x = np.linspace(laser_model.ppf(0.01),laser_model.ppf(0.99), 100)
ax.plot(x, laser_model.pdf(x),'k', lw=3, label='Laser Model PDF')
lots_of_samples = laser_model.rvs(1000)
ax.hist(lots_of_samples, density=True, histtype='stepfilled', alpha=0.2)
plt.show()

In [ ]:
uni_dist = stats.uniform(loc=-1, scale=2)
fig, ax = plt.subplots(1, 1)
#fig.set_size_inches((5,4))
x = np.linspace(uni_dist.ppf(0.01), uni_dist.ppf(0.99),100)
ax.plot(x, uni_dist.pdf(x),'k',lw=3)
uni_samples = uni_dist.rvs(1000)
#fig.set_size_inches((5,4))
ax.hist(uni_samples, density=True, histtype='stepfilled', alpha=0.2)
plt.show()

## PDFs for Sensor Modeling
Basic sensor model: a **range-bearing** sensor.
This sensor returns the distance and angle from a detector's $x$-axis for an object within its cone of detection.

In [ ]:
from math import atan2, atan

def r_2d(x_obj, x_s):
    return linalg.norm(x_obj-x_s)

def bearing_2d(x_obj, x_s):
    return atan2((x_obj[1]-x_s[1]), (x_obj[0]-x_s[0]))

In [ ]:
Sigma = np.array([[0.05, 0.0],[0.0,0.01]])

In [ ]:
# Example noise with the covariance matrix Sigma
noise_samples = stats.multivariate_normal(cov=Sigma).rvs(1000)
fig, ax = plt.subplots(1, 1)
fig.set_size_inches((6,4))
# This scatter plot is in the dimensions of r and theta, not the 2d plane
# Note the scales are different since the covariances differ by an order of magnitude
ax.scatter(noise_samples[:,0], noise_samples[:,1])
plt.show()

In [ ]:
# A function that models a range-bearing measurement based on 
# system position, x_t, and the detected object, x_obj
def measure(x_obj, x_t):
    h = np.array([r_2d(x_obj, x_t), bearing_2d(x_obj, x_t)])
    return stats.multivariate_normal(mean=h, cov=Sigma).rvs()

In [ ]:
xo = np.array([1,1])
xs = np.array([0,0])

In [ ]:
print("Exact measurement: {0}".format([r_2d(xo, xs), bearing_2d(xo, xs)]))
print("Noisy measurement: {0}".format(measure(xo, xs)))

In [ ]:
ps = np.array([measure(xo, xs) for i in range(1000)])
xlocs = ps[:,0]*np.cos(ps[:,1])
ylocs = ps[:,0]*np.sin(ps[:,1])

In [ ]:
fig, ax = plt.subplots(1, 1)
fig.set_size_inches((7,5))
ax.scatter(xlocs,ylocs, marker='o')
ax.set(xlim=(0,2), ylim=(0,2))
plt.show()